# Première implementation des modèles

Ce notebook est fait pour faire des tests sur l'implémentation des modèles, de la génération des données, des graphiques, etc.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import particles
import seaborn as sns
from particles.state_space_models import Bootstrap

from smc_movement_models.models import MarineSSM
from smc_movement_models.plots import plot_graph_values, plot_real_data

np.random.seed(42)

In [ ]:
gv_fig = plot_graph_values(figsize=None)

In [ ]:
day_fig = plot_real_data("../data/19_01_2008.csv", figsize=(10, 5))

In [ ]:
day = pd.read_csv("../data/19_01_2008.csv", parse_dates=[0])

In [ ]:
window = day[:312]["Velocity"]

In [ ]:
def run_smc(window, N=200, **kwargs):
    my_ssm_model = MarineSSM(z0=window[0], z1=window[1], **kwargs)
    my_fk_model = Bootstrap(ssm=my_ssm_model, data=window)
    my_alg = particles.SMC(fk=my_fk_model, N=N, store_history=True)
    my_alg.run()
    return my_alg


def estimate_a1_a2_on_window(window, N=200, M=10, epsilon=0.1, alpha=0.5, **kwargs):
    final_a1s = []
    final_a2s = []
    for m in range(M):
        a1 = final_a1s[-1].mean() if m > 0 else 0
        a2 = final_a2s[-1].mean() if m > 0 else 0
        alg = run_smc(
            window=window,
            sigma_v=epsilon,
            a1=a1,
            a2=a2,
            sigma_o=0.1,
            sigma_e=0.1,
            c1=0.9,
            c2=0.1,
            delta=10,
        )
        a1s_tmp = np.array(alg.hist.X)[:, :, 2]
        a2s_tmp = np.array(alg.hist.X)[:, :, 3]
        wgts = np.array([w.W for w in alg.hist.wgts])
        final_a1s.append(np.average(a1s_tmp, weights=wgts, axis=1))
        final_a2s.append(np.average(a2s_tmp, weights=wgts, axis=1))
        epsilon *= 0.5

    final_a1s = np.array(final_a1s)
    final_a2s = np.array(final_a2s)
    a1s = final_a1s.mean(axis=1)
    a2s = final_a2s.mean(axis=1)
    return a1, a2

In [ ]:
epsilon = 0.1
final_a1s = []
final_a2s = []
for m in range(10):
    a1 = final_a1s[-1].mean() if m > 0 else 0
    a2 = final_a2s[-1].mean() if m > 0 else 0
    alg = run_smc(window=window, sigma_v=epsilon, a1=a1, a2=a2)
    a1s_tmp = np.array(alg.hist.X)[:, :, 2]
    a2s_tmp = np.array(alg.hist.X)[:, :, 3]
    wgts = np.array([w.W for w in alg.hist.wgts])
    final_a1s.append(np.average(a1s_tmp, weights=wgts, axis=1))
    final_a2s.append(np.average(a2s_tmp, weights=wgts, axis=1))
    epsilon *= 0.5

final_a1s = np.array(final_a1s)
final_a2s = np.array(final_a2s)
a1s = final_a1s.mean(axis=1)
a2s = final_a2s.mean(axis=1)

In [ ]:
print(a1s, a2s)

In [ ]:
for i in range(10):
    sns.kdeplot(np.array(final_a1s)[i, :], fill=True, label=f"iter {i+1}")

plt.legend()

In [ ]:
my_alg = run_smc(window, N=200)

In [ ]:
a1s_tmp = np.array(alg.hist.X)[:, :, 2]
a2s_tmp = np.array(alg.hist.X)[:, :, 3]
wgts = np.array([w.W for w in alg.hist.wgts])
a1s = np.average(a1s_tmp, weights=wgts, axis=1)
a2s = np.average(a2s_tmp, weights=wgts, axis=1)

In [ ]:
sns.kdeplot(a1s, fill=True, label="a1")
sns.kdeplot(a2s, fill=True, label="a2")
plt.legend()

In [ ]:
x_range = np.arange(len(window))
fig, axs = plt.subplots(2, 1, figsize=(10, 5))
axs[0].plot(x_range, a1s)
axs[1].plot(x_range, a2s)